In [64]:
import numpy as np
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

In [65]:
def splitWords(t):
    splitTweet = []
    t = t.split(' ')
    
    for word in t:
        splitTweet.extend(re.split(r'[|,;!|:]', word))
    
    return ' '.join(splitTweet)

In [66]:
def checkWord(token):
    fillers = ['link', 'http']
    return (not token.is_stop) and (token.is_alpha) and (token.text not in fillers) and (len(token.text) > 3)

In [67]:
# What about negations like "not"?
# Contractions like "don't" or "can't"?
# Don't include names of people or cities
# Remove slang like hmmm
# Filter out swear words

# Split words by where capitalized letter is

In [68]:
### Source: https://spacy.io/usage/linguistic-features

def spacyPipeline(tweets):
    nlp = spacy.load('en_core_web_sm')
    
    indices = []
    preprocessed_tweets = []
    for index, t in enumerate(tweets):
        
        doc = nlp(t)
        filtered_tweet = set()
        
        for token in doc:
            # print(token, " | ", spacy.explain(token.pos_))
            if (token.lemma_ not in filtered_tweet) and checkWord(token):
                    filtered_tweet.add(token.lemma_.lower())
        
        if len(filtered_tweet) >= 4:
            # print(filtered_tweet,"\n---\n")
            preprocessed_tweets.append(filtered_tweet)
            indices.append(index)
    
    return preprocessed_tweets, indices

In [69]:
def convertClasses(c, indices):
    classes = []
    indices = set(indices)

    neg = 0
    pos = 1
    
    for index in indices:
        if pd.isnull(c[index]) or c[index] == 'N' or c[index] == 'No':
            classes.append(neg)
        else:
            classes.append(pos)
    
    return classes

In [70]:
def preprocess(data):
    tweets = data['tweet']
    
    # Convert all to lowercase
    tweets = [splitWords(t) for t in tweets]
    
    # Process tweets through spaCy pipeline
    tweets, indices = spacyPipeline(tweets)
    
    # Transform with TF-IDF
    transformer = TfidfVectorizer()
    tfidf_tweets = transformer.fit_transform([' '.join(t) for t in tweets])
    
    # Transform classes
    classes = convertClasses(data["existence"].tolist(), indices)
    
    return tfidf_tweets, classes